In [1]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error


#### unsqueez() 함수 이해하기

In [12]:
# 지정한 위치에 차원하나를 추가
# 예를 들어 2,2 텐서에 unsqueeze(0)를 수행하면 텐서의 맨 앞에 차원을 추가 => (1,2,2)
tensor1 = torch.Tensor(np.ones((2,2)))
print("Original:")
print(tensor1)
print(tensor1.size())
print("unsqueeze(0): ")
usqz0 = tensor1.unsqueeze(0) 
print(usqz0)
print(usqz0.size())
print("unsqueeze(1): ")
usqz1 = tensor1.unsqueeze(1)
print(usqz1)
print(usqz1.size())
print("unsqueeze(2): ")
usqz2 = tensor1.unsqueeze(2)
print(usqz2)
print(usqz2.size())

Original:
tensor([[1., 1.],
        [1., 1.]])
torch.Size([2, 2])
unsqueeze(0): 
tensor([[[1., 1.],
         [1., 1.]]])
torch.Size([1, 2, 2])
unsqueeze(1): 
tensor([[[1., 1.]],

        [[1., 1.]]])
torch.Size([2, 1, 2])
unsqueeze(2): 
tensor([[[1.],
         [1.]],

        [[1.],
         [1.]]])
torch.Size([2, 2, 1])


#### 5.1 다층 퍼셉트론

##### 5.1.1 선형회귀

In [2]:
class LinearRegress(nn.Module):
    def __init__(self):
        super().__init__()
        # 선형회귀 모델 y = w*x+b 표현 입력: 1, 출력: 1
        self.fc = nn.Linear(1, 1, bias=True)
    def forward(self, x):
        y = self.fc(x)
        return y
        

In [15]:
x = torch.FloatTensor(range(15)).unsqueeze(1) # 입력 값
y = 2*x+torch.rand(15,1) # 정답

model = LinearRegress()
learning_rate = 1e-3
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

loss_stack = []
for epoch in range(50):
    # Pytorch는 기본설정에 옵티마이저의 gradients값들을 추후 역전파 시 가산시키도록 되어 있음,
    # 이러한 미분값 가산방식은 RNN을 학습시킬 때 편리함, 그러나 기본적인 방식의 학습일 경우
    # 학습이 한번 종료되면 반드시 옵티마이저의 이전 미분값을 zero_grad()로 초기화 하여 학습을 진행햐야 함
    optimizer.zero_grad() # 최적화는 계산을 누적시키기 때문에 매 epoch 마다 zero_grad()를 통해 값을 초기화 해준다.
    y_hat = model.forward(x)
    loss = criterion(y, y_hat)    
    loss.backward()
    optimizer.step()
    loss_stack.append(loss.item())

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.plot(loss_stack)
plt.title("Loss")
plt.subplot(122)
plt.plot(y.numpy(), 'r.')
plt.title("Prediction")
plt.plot(y_hat.detach().numpy(), 'b-') # grad(미분값)이 포함된 Tensor를 넘파이로 변환하기전에 detach()로 grad제거
plt.show()
    

KeyboardInterrupt: 

##### 5.1.2 집값 예측하기

In [4]:
# 데이터 셋 클래스
class TensorData(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = y_data.shape[0]
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    def __len__(self):
        return self.len

In [5]:
# 데이터 셋 만들기
df = pd.read_csv("./data/reg.csv", index_col=[0])
# 데이터 프레임을 넘파일 배열로
X = df.drop('Price', axis=1).to_numpy()
Y = df['Price'].to_numpy().reshape(-1, 1)
# 훈련 데이터와 테스트 데이터 나누기 
#  * 순서는 1)훈련X, 2)테스트X, 3)훈련Y, 4)테스트X
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.7)

In [6]:
# 텐서 데이터 만들기
trainset = TensorData(X_train, Y_train)
trainLoader = DataLoader(trainset, batch_size=32, shuffle=True)
testset = TensorData(X_test, Y_test)
testLoader = DataLoader(testset, batch_size=32, shuffle=False)

In [11]:
# 모델 구축
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 1)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = F.relu(self.fc1(x))        
        x = self.dropout(F.relu(self.fc2(x)))        
        x = F.relu(self.fc3(x))
        return x


In [14]:
# 모델 학습하기
model = Regressor()
criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-7)
# weight decay는 weight들의 값이 증가하는 것을 제한함으로써, 
# 모델의 복잡도를 감소시킴으로써 제한하는 기법, 
# 즉 weight를 decay(부식시킨다라는 의미를 조금 감량시키는 의미)시켜서 
# Overfitting을 방지하는 기법
# SGD 경우 Regualarization: (1-에타^람다)W - 에타*미분(MSE)
# Regularization 수식에서 람다값이 weight_decay값임!

loss_ = []
minibatch_cnt = len(trainLoader) # 데이터로더에 포함된 미니배치의 갯수
# ex) 데이터수: 1000, 배치사이즈: 25 => minibatch_cnt: 40 => 미니배치 데이터의 개수
for i in range(400):
    running_loss = 0.0
    for data in trainLoader:
        inputs, values = data
        optimizer.zero_grad()
        y = model.forward(inputs)
        e = criterion(y, values)
        e.backward()
        optimizer.step()
        running_loss += e.item()
    epoch_loss = running_loss/minibatch_cnt
    loss_.append(epoch_loss)
    if i % 40 == 0:
        print(f"Epoch: {i}, Loss: {epoch_loss}")
        
plt.plot(loss_)
plt.show()



KeyboardInterrupt: 

In [12]:
# 모델 평가하기
def evaluateModel(dataloader):
    predictions = torch.tensor([], dtype=torch.float)
    actuals = torch.tensor([], dtype=torch.float)
    with torch.no_grad(): # require_grad를 비활성화 시킴 => 역전파를 위해 사용하는 연산을 비활성화
        model.eval() # 드롭아웃과 같이 과적합 방지를 위한 정규화 기법을 비활성화 시킴
        for data in dataloader:
            inputs, values = data
            output = model.forward(inputs)
            # predictions에 output 데이터를 누적시킴
            # output 데이터가 (1, 32) 경우 axis=0으로 누적되면
            # (1, 32) -> (2, 32) 처럼 0번째 차원을 기준으로 데이터가 누적됨
            predictions = torch.cat((predictions, output), 0)
            actuals = torch.cat((actuals, values), 0)
    predictions = predictions.numpy()
    actuals = actuals.numpy()
    rmse = np.sqrt(mean_squared_error(predictions, actuals))
    return rmse



In [13]:
train_loss = evaluateModel(trainLoader)
test_loss = evaluateModel(testLoader)

print(f"Train RMSE: {train_loss}, Test RMSE: {test_loss}")

Train RMSE: 0.08908578753471375, Test RMSE: 0.10730893164873123


#### 5.5 교차검증

##### 5.5.1 교차 검증을 통한 집값 예측 모델 평가

In [22]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=3, shuffle=True)

In [25]:
validation_loss = []
model = Regressor()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-7)

for fold, (train_idx, val_idx) in enumerate(kfold.split(trainset)):
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subdampler = torch.utils.data.SubsetRandomSampler(val_idx)
    trainloader = DataLoader(trainset, batch_size=32, sampler=train_subsampler)
    valloader = DataLoader(trainset, batch_size=32, sampler=val_subdampler)

    trainloss_stack = []
    valloss_stack = []
    for i in range(400):
        #  훈련데이터로 학습수행
        minibatch_cnt = len(trainloader)
        minibatch_loss = 0.0
        for data in trainloader:
            inputs, values = data
            model.zero_grad()
            outputs = model.forward(inputs)
            #loss = mean_squared_error(values, outputs)
            loss = criterion(outputs, values)
            loss.backward()
            optimizer.step()
            minibatch_loss += loss.item()
        trainloss_stack.append( np.sqrt( minibatch_loss/minibatch_cnt) )

    train_loss = np.array(trainloss_stack).mean()
    # 검증 데이터로 평가 수행
    val_loss = evaluateModel(valloader)

    print(f"Kfold: {fold}, train loss: {train_loss}, validation loss:{val_loss}")

test_loss = evaluateModel(testLoader)
print(f"Test loss: {test_loss}")

Kfold: 0, train loss: 0.10626403144474075, validation loss:0.11219097673892975
Kfold: 1, train loss: 0.060652537166520124, validation loss:0.08319053798913956
Kfold: 2, train loss: 0.0423403420423471, validation loss:0.08519025892019272
Test loss: 0.1447765827178955


#### 5.6 모델 구조 및 가중치 확인

#### 5.6.1 모델구조

In [26]:
from torchsummary import summary

In [27]:
print(model)

Regressor(
  (fc1): Linear(in_features=13, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=30, bias=True)
  (fc3): Linear(in_features=30, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


##### 5.6.2 모델변수

In [28]:
for parameter in model.parameters():
    print(parameter.size())

torch.Size([50, 13])
torch.Size([50])
torch.Size([30, 50])
torch.Size([30])
torch.Size([1, 30])
torch.Size([1])


In [32]:
print(model.fc1.weight.size(), model.fc1.bias.size())

torch.Size([50, 13]) torch.Size([50])


In [33]:
for name, param in model.named_parameters():
    print(name, param.size())

fc1.weight torch.Size([50, 13])
fc1.bias torch.Size([50])
fc2.weight torch.Size([30, 50])
fc2.bias torch.Size([30])
fc3.weight torch.Size([1, 30])
fc3.bias torch.Size([1])


In [35]:
summary(model, (10, 13))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 10, 50]             700
            Linear-2               [-1, 10, 30]           1,530
           Dropout-3               [-1, 10, 30]               0
            Linear-4                [-1, 10, 1]              31
Total params: 2,261
Trainable params: 2,261
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.01
Estimated Total Size (MB): 0.02
----------------------------------------------------------------
